In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4090/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_6_9_10,0.998100,0.968097,0.996466,0.988454,0.559519,2.392947,0.696834,1.594774,"Hidden Size=[20], regularizer=0.1, learning_ra..."
1,model_6_9_11,0.998098,0.967291,0.996283,0.988099,0.560070,2.453451,0.732855,1.643752,"Hidden Size=[20], regularizer=0.1, learning_ra..."
2,model_6_9_9,0.998094,0.968967,0.996659,0.988833,0.561285,2.327720,0.658842,1.542367,"Hidden Size=[20], regularizer=0.1, learning_ra..."
3,model_6_9_12,0.998090,0.966546,0.996111,0.987769,0.562301,2.509325,0.766790,1.689310,"Hidden Size=[20], regularizer=0.1, learning_ra..."
4,model_6_9_13,0.998079,0.965859,0.995950,0.987464,0.565709,2.560811,0.798508,1.731495,"Hidden Size=[20], regularizer=0.1, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...
1051,model_16_6_3,0.961026,0.934343,0.943534,0.938729,11.475258,19.755845,17.449082,18.670297,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
1066,model_12_8_0,0.960510,0.932522,0.942428,0.939960,11.627212,9.591927,13.998252,11.665495,"Hidden Size=[35], regularizer=0.1, learning_ra..."
1079,model_15_8_1,0.960017,0.944171,0.973861,0.961380,11.772395,9.547010,6.921434,8.311448,"Hidden Size=[30, 15], regularizer=0.01, learni..."
1102,model_10_4_2,0.958614,0.953334,0.928650,0.946264,12.185399,16.630589,13.321542,15.073393,"Hidden Size=[20, 10], regularizer=0.1, learnin..."
